# Silver Data Transformation 

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df=spark.read.format("delta")\
.option("header",True)\
.option("inferSchema",True)\
.load("abfss://bronze@netflixprojectdlmani.dfs.core.windows.net/netflix_titles")

In [0]:
df=df.fillna({"duration_minutes":0,"duration_seasons":1})

In [0]:
df = df.withColumn("duration_minutes", regexp_replace("duration_minutes", "\"", ""))\
    .withColumn("duration_minutes",regexp_replace(col("duration_minutes"),"Flying Fortress", "0"))\
 .withColumn("duration_minutes",regexp_replace(col("duration_minutes"),"and probably will.","0"))\
     .withColumn("duration_seasons",regexp_replace(col("duration_seasons"),"3/31/2017","0"))

In [0]:
df.display()

In [0]:
df=df.withColumn("duration_minutes",col('duration_minutes').cast(IntegerType()))\
.withColumn("duration_seasons",col('duration_seasons').cast(IntegerType()))

In [0]:
df=df.withColumn("ShortTitle",split(col('title'),':')[0])
df.display()

In [0]:
df=df.withColumn("rating",split(col("rating"),"-")[0])
df.display()

In [0]:
df=df.withColumn("type_flag",when(col('type')=="Movie",1)\
    .when(col('type')=="TV Show",2)\
    .otherwise(0))
df.display()

In [0]:
from pyspark.sql.window import Window

In [0]:
df=df.withColumn("duration_ranking",dense_rank().over(Window.orderBy(col("duration_minutes").desc())))

In [0]:
df.display()

In [0]:
df_vis=df.groupBy("type").agg(count("*" ).alias("Totalcount"))
df_vis.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df.display()

In [0]:
df.write.format("delta")\
    .mode("overwrite")\
    .option("path","abfss://silver@netflixprojectdlmani.dfs.core.windows.net/netflis_titles").save()